<a href="https://colab.research.google.com/github/d-m-bailey/ihp-mpw-be/blob/main/TO_202504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend Verification for the iHP April 2025 Open MPW Shuttle

[D. Mitch Bailey](https://www.linkedin.com/in/mitch-bailey-cvc/) [ShuhariSystem](https://www.shuharisystem.com/)

# Overview
This notebook performs backend LVS (with [klayout](https://github.com/KLayout/klayout) or [magic](https://github.com/RTimothyEdwards/magic)/[netgen](https://github.com/RTimothyEdwards/netgen)), soft-conneciton checks (with magic/netgen) and/or reliability verfication (with [CVC-RV](https://github.com/d-m-bailey/cvc)) with open source EDA software and systems.

# iHP Open-MPW Data
The 17 submissions to iHp open mpw shuttle for April 2025 listed below are located [here](https://github.com/IHP-GmbH/TO_Apr2025).

* 160GHz_LNA
* 40_GHZ_LOW_NOISE_TIA
* 6502-cpu
* 97_GHZ_LINEAR_TIA
* DC_to_130_GHz_TIA
* GPS_LNA
* Greyhound
* Mixer5GHz
* PA_180GHz
* TTIHP0p2
* TTIHP25a
* VCO_130nm_LSI
* active_L_VCOs
* ascon
* bandgap_ref_cmos
* elemrv-n
* i2c-gpio-expander



In [ ]:
%%writefile /content/env
export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
echo $PATH
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export EXTRA_CHECK_COMMIT=main
export CVC_COMMIT=master
#export PRECHECK_ROOT=/content/mpw_precheck
#export CARAVEL_ROOT=/content/caravel
export UPRJ_ROOT=/content/data
export LVS_ROOT=/content/extra_be_checks

export MPW=TO_Apr2025
export SLOT=slot-002
export TOP=user_project_wrapper
unset WORK_ROOT

# User setup
Only needs to be executed once.

This step sets up the pdk and installs magic, klayout, netgen and cvc_rv.

In [ ]:
%%shell
cd
cat /content/env
source /content/env
#pip install udocker
if [[ ! -d $PDK_ROOT ]]; then
  echo "==> Installing ciel..."
  pip install ciel
  #udocker --allow-root install
  #rm -rf mpw_precheck
  #git clone https://github.com/efabless/mpw_precheck.git $PRECHECK_ROOT
  ciel enable --pdk $PDK $PDK_COMMIT
  echo "==> Using $PDK $PDK_COMMIT"
fi

if [[ ! -d extra_be_checks ]]; then
  echo "==> Downloading extra_be_checks $EXTRA_CHECK_COMMIT"
  rm -rf extra_be_checks
  git clone https://github.com/d-m-bailey/extra_be_checks.git -b $EXTRA_CHECK_COMMIT
  #adduser lvs
  #usermod -aG sudo lvs
fi

if [[ ! -d netgen ]]; then
  echo "==> Downloading and installing netgen $NETGEN_COMMIT"
  git clone https://github.com/RTimothyEdwards/netgen.git --depth=1 -b $NETGEN_COMMIT
  cd netgen
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
netgen -batch

if [[ ! -d magic ]]; then
  echo "==> Downloading and installing magic $MAGIC_COMMIT"
  git clone https://github.com/RTimothyEdwards/magic.git --depth=1 -b $MAGIC_COMMIT
  cd magic
  autoreconf -vif
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
magic -dnull -noc --version

if [[ ! -d cvc ]]; then
  echo "==> Downloading and installing cvc_rv $CVC_COMMIT"
  sudo get-apt install autopoint
  git clone https://github.com/d-m-bailey/cvc --depth=1 -b $CVC_COMMIT
  cd cvc
  autoreconf -vif
  ./configure --prefix=$LOCAL_INSTALL --disable-nls
  make
  make install
  cd
fi
cvc_rv -v

Create a script to extract the user_project_wrapper hierarchy as gds from the caravel oasis.

Only needs to be run once.

In [ ]:
%%writefile oas2gds.py
import pya

app = pya.Application.instance()
opt = pya.SaveLayoutOptions()
layout_view = pya.Layout()

input_layout = "caravel.oas"
# Setting the name of the output file and setting the substitution character
print("[INFO] Reading from " + input_layout)

# Reading the input file and writing it to the output file name
layout_view.read(input_layout)
for cell_it in layout_view.each_cell():
    if cell_it.name.endswith("project_wrapper"):
        myIndex = layout_view.cell(cell_it.name).cell_index()
        output = "work/gds/" + cell_it.name + ".gds.gz"
        opt.set_format_from_filename(output)
        opt.oasis_substitution_char=''
        break
opt.select_cell(myIndex)
print("[INFO] Writing " + output)
layout_view.write(output, opt)

app.exit(0)

# Clone the design repo and create gds.

In [ ]:
%%shell
cat /content/env
source /content/env
rm -rf data
git clone https://foss-eda-tools.googlesource.com/third_party/shuttle/sky130/$MPW/$SLOT.git data
rm -rf data/.git data/tapeout/outputs/gds
if [[ -d oas ]]; then
  ln -s $PWD/data/oas/cara*oas data/caravel.oas
else
  ln -s $PWD/data/tapeout/outputs/oas/cara*_*oas data/caravel.oas
fi
mkdir -p data/work/gds
cd data
cp ../oas2gds.py .
chown -R lvs:lvs .
sudo -u lvs udocker run -i -t -v $PWD:$PWD \
  -u $(id -u lvs):$(id -g lvs) \
  efabless/mpw_precheck:latest bash -c "cd $PWD ; klayout -b -rm oas2gds.py"

# List verilog and spice files

In [ ]:
%%shell
source /content/env
cd data
find . -name '*.spice'
find . -name '*.v'

# Create lvs_config.json

Update the TOP_LAYOUT, LVS_SPICE_FILES, LVS_VERILOG_FILES, and LAYOUT_FILE for every design.

Update the other parameters as needed.

In [ ]:
%%writefile lvs_config.json
{
        "STD_CELL_LIBRARY": "sky130_fd_sc_hd",
        "INCLUDE_CONFIGS": [
                "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
        ],
        "TOP_SOURCE": "$TOP",
        "TOP_LAYOUT": "SJ_user_project_wrapper",
        "EXTRACT_FLATGLOB": [
                ""
        ],
        "EXTRACT_ABSTRACT": [
                ""
        ],
        "LVS_FLATTEN": [
                ""
        ],
        "LVS_NOFLATTEN": [
                ""
        ],
        "LVS_IGNORE": [
                ""
        ],
        "LVS_SPICE_FILES": [
                ""
        ],
        "LVS_VERILOG_FILES": [
                "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
                "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
        ],
        "LAYOUT_FILE": "$UPRJ_ROOT/work/gds/SJ_user_project_wrapper.gds.gz"
}

# Run BE checks

In [ ]:
%%shell
cat /content/env
source /content/env
cd data
gdspath=$(ls work/gds/*project_wrapper.gds.gz)
gdsfile=${gdspath##*/}
layout=${gdsfile%%.gds.gz}
source=${layout//[A-Z0-9][A-Z0-9]_}
echo "Layout is $layout, source is $source"
cp $LVS_ROOT/tech/$PDK/cvc.power.$source work/cvc.power.$layout
cp ../lvs_config.json lvs_config.json
export INPUT_DIRECTORY=$PWD
sudo -u lvs udocker run -i -t -v $PRECHECK_ROOT:$PRECHECK_ROOT \
  -v $INPUT_DIRECTORY:$INPUT_DIRECTORY \
  -v $LVS_ROOT:$LVS_ROOT \
  -v $PDK_ROOT:$PDK_ROOT \
  -e INPUT_DIRECTORY=$INPUT_DIRECTORY \
  -e PDK_PATH=$PDK_ROOT/$PDK \
  -e PDK=$PDK \
  -e TOP=$TOP \
  -e PDK_ROOT=$PDK_ROOT \
  -e UPRJ_ROOT=$UPRJ_ROOT \
  -e PDKPATH=$PDKPATH \
  -e LVS_ROOT=$LVS_ROOT \
  -e WORK_ROOT=$INPUT_DIRECTORY/work \
  -u $(id -u lvs):$(id -g lvs) \
  efabless/mpw_precheck:latest bash -c "env; cd $INPUT_DIRECTORY ; $LVS_ROOT/run_be_checks lvs_config.json"
gunzip -f work/cvc.error.gz
head -1 work/cvc.oeb.report > work/cvc.oeb.sort.report
tail -n +2 work/cvc.oeb.report | sort -k1,1n >> work/cvc.oeb.sort.report

# View verification logs

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.log')
files.view('/content/data/work/lvs.log')
files.view('/content/data/work/cvc.log')
files.view('/content/data/work/cvc.oeb.log')

# View soft verification report

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.report')

# View LVS report

In [ ]:
from google.colab import files

files.view('/content/data/work/lvs.report')

# View CVC report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.error')

# View OEB report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.oeb.sort.report')